# Weather Prediction Model
This notebook will contain a ML model that predicts comfortable temperatures based on historical data. For this model, I'll be using San Jose's weather data from 2010 to June 28th, 2020. The reason for using as much data is that the variety in the data can help avoid overfitting.

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sanJoseWeather = pd.read_csv('WeatherData/Template/sanJose_Weather_Solar.csv')

sanJoseWeather.head()

,Year,Month,Day,Max Temp (F),Min Temp (F),Average Wind Speed (mph),Relative Humidity (%)
0,2010,1,1,63.0,49.0,2.3,94
1,2010,1,2,58.0,45.0,2.4,96
2,2010,1,3,60.0,39.0,3.4,89
3,2010,1,4,57.0,42.0,2.8,86
4,2010,1,5,59.0,38.0,2.3,90


In [69]:
from math import sqrt, fabs, pow, floor

############
def rothfusz_heat_index_regression(temperature, relative_humidity):
    """[Computes the 'feels like' temperature based on Rothfusz's regression]

    Args:
        temperature (float): Temperature in Fahrenheit
        relative_humidity (float): Relative humidity in percentage

    Returns:
        float: Adjusted heat index temperature
    """
    T, RH = temperature, relative_humidity

    added_adjustment = ((RH - 85)/10) * ((87 - T)/5)
    subtracted_adjustment = ((13 - RH)/4) * sqrt(17 - fabs(T - 95) / 17)

    heat_index = (-42.379) + (2.04901523)*(T) + (10.14333127)*(RH) - (0.22475541)*(T)*(RH)
    heat_index = heat_index - (0.00683783)*(T)*(T) - (0.05481717)*(RH)*(RH) + (0.00122874)*(T)*(T)*(RH)
    heat_index = heat_index + (0.00085282)*(T)*(RH)*(RH) - (0.00000199)*(T)*(T)*(RH)*(RH)

    if ((RH < 13) and (80 < T < 112)):
        heat_index -= subtracted_adjustment
    elif ((RH > 85) and (80 < T < 87)):
        heat_index += added_adjustment

    return round(heat_index)

def steadman_heat_index_regression(temperature, relative_humidity):
    """[Computes the 'feels like' temperature based on Steadman's regression]

    Args:
        temperature (float): Temperature in Fahrenheit
        relative_humidity (float): Relative humidity in percentage

    Returns:
        float: Adjusted heat index temperature
    """
    T, RH = temperature, relative_humidity
    heat_index = (0.5) * (T + (61.0) + ((T - 68) * 1.2) + (RH * (0.094)))
    
    return round(heat_index)

def heat_index(temperature, relative_humidity):
    """[Computes the 'feels like' temperature for usage]

    Args:
        temperature (float): Temperature in Fahrenheit
        relative_humidity (float): Relative humidity in percentage

    Returns:
        float: Adjusted heat index temperature based on NOAA's recommendation
    """
    T, RH = temperature, relative_humidity
    
    temp = steadman_heat_index_regression(temperature=T, relative_humidity=RH)
    steadman = (0.5) * (temp + T)

    rothfusz = rothfusz_heat_index_regression(temperature=T, relative_humidity=RH)

    if (steadman >= 80):
        return rothfusz
    
    return round(steadman)

################

def windchill_regression(temperature, wind_speed):
    T, V = temperature, wind_speed
    feels_like = 35.74 + (0.6215) * (T) - (35.75) * (pow(V, 0.16)) + (0.4275 * T) * (pow(V, 0.16))
    return round(feels_like)

def get_windchill(temperature, wind_speed):
    T, V = temperature, wind_speed
    if (T > 50):
        return temperature
    else:
        return windchill_regression(T, V)

####################

def real_feel(temperature, relative_humidity, wind_speed):
    T, RH, WS = temperature, relative_humidity, wind_speed
    if (temperature > 50):
        return heat_index(temperature=T, relative_humidity=RH)
    return get_windchill(temperature=T, wind_speed=WS)

In [70]:
sanJoseWeather.isna().sum()

Year                         0
Month                        0
Day                          0
Max Temp (F)                32
Min Temp (F)                34
Average Wind Speed (mph)     0
Relative Humidity (%)        0
dtype: int64

In [71]:
sanJoseWeather['Max Temp (F)'].fillna(round(sanJoseWeather['Max Temp (F)'].mean()), inplace=True)
sanJoseWeather['Min Temp (F)'].fillna(round(sanJoseWeather['Min Temp (F)'].mean()), inplace=True)

In [72]:
sanJoseWeather.isna().sum()

Year                        0
Month                       0
Day                         0
Max Temp (F)                0
Min Temp (F)                0
Average Wind Speed (mph)    0
Relative Humidity (%)       0
dtype: int64

In [73]:
sanJoseWeather.insert(loc=7, column='Real Feel (F)', value=0, allow_duplicates=False)

In [74]:
sanJoseWeather['Real Feel (F)'] = sanJoseWeather.apply(lambda row: real_feel(row['Max Temp (F)'], row['Relative Humidity (%)'], row['Average Wind Speed (mph)']), axis=1)

In [75]:
sanJoseWeather.head()

,Year,Month,Day,Max Temp (F),Min Temp (F),Average Wind Speed (mph),Relative Humidity (%),Real Feel (F)
0,2010,1,1,63.0,49.0,2.3,94,63
1,2010,1,2,58.0,45.0,2.4,96,58
2,2010,1,3,60.0,39.0,3.4,89,60
3,2010,1,4,57.0,42.0,2.8,86,56
4,2010,1,5,59.0,38.0,2.3,90,59


In [76]:
sanJoseWeather.isna().sum()

Year                        0
Month                       0
Day                         0
Max Temp (F)                0
Min Temp (F)                0
Average Wind Speed (mph)    0
Relative Humidity (%)       0
Real Feel (F)               0
dtype: int64

In [77]:
import random
from time import gmtime
from calendar import timegm

def user_heating(windchill_temperature):
    # Gets the systems milliseconds since the epoch
    mseconds = timegm(gmtime())
    # Sets the milliseconds as the random seed for generation
    rand_seed = random.seed(mseconds * random.random())

    min_heat = 60
    avg_heat = 66
    max_heat = 71
    if (windchill_temperature <= 45):
        return random.randrange(avg_heat, max_heat, 1)
    elif (windchill_temperature > 45):
        return random.randrange(min_heat, avg_heat, 1)

def user_cooling(feels_like_temperature):
    # Gets the systems milliseconds since the epoch
    mseconds = timegm(gmtime())
    # Sets the milliseconds as the random seed for generation
    rand_seed = random.seed(mseconds * random.random())

    min_cool = 74
    avg_cool = 79
    max_cool = 85
    if (feels_like_temperature >= 90):
        return random.randrange(min_cool, avg_cool, 1)
    elif (feels_like_temperature < 90):
        return random.randrange(avg_cool, max_cool, 1)

def user_normal(temperature):
    return floor(temperature)

def get_user_data(weighted_temperature):
    if (weighted_temperature <= 64):
        # The WHO recommends a minimum indoor temperature of 64 degrees Fahrenheit
        return user_heating(windchill_temperature=weighted_temperature) # Run the heat
    elif (weighted_temperature > 64 and weighted_temperature <= 80):
        return user_normal(weighted_temperature) # The house is at room temperature
    elif (weighted_temperature > 80):
        return user_cooling(feels_like_temperature=weighted_temperature) # Otherwise run the A/C

In [78]:
sanJoseWeather.insert(loc=8, column='Users Thermostat (F)', value=0, allow_duplicates=False)

In [79]:
sanJoseWeather['Users Thermostat (F)'] = sanJoseWeather.apply(lambda row: get_user_data(row['Real Feel (F)']), axis=1)

In [80]:
sanJoseWeather.isnull().sum()

Year                        0
Month                       0
Day                         0
Max Temp (F)                0
Min Temp (F)                0
Average Wind Speed (mph)    0
Relative Humidity (%)       0
Real Feel (F)               0
Users Thermostat (F)        0
dtype: int64

In [85]:
sanJoseWeather.to_csv(path_or_buf='WeatherData/Template/sanJoseComplete_User.csv')